In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
# you can find the latest package @ https://github.com/kubeflow/pipelines/releases
#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

In [3]:
EXPERIMENT_NAME = 'Hellow world!'
BASE_IMAGE = 'ubuntu:16.04'
LOCAL_PATH = '/mnt/volume'
PVC_NAME = 'nfs2'

# Using Local Volumes and NFS

Reference documentation: https://www.kubeflow.org/docs/gke/cloud-filestore/

This pipeline demonstrates using Cloud Filestore with Kubeflow

In [4]:
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret
from kubernetes import client as k8s_client
from kfp import compiler
from kfp import notebook
from kfp import components as comp
from kfp.onprem import mount_pvc

In [5]:
class WriteOp(dsl.ContainerOp):
    def __init__(self, msg):
        super(WriteOp, self).__init__(
          name='write',
          image=BASE_IMAGE,
          command=['/bin/bash'],
          arguments= ['-c', 'echo ' + str(msg) + ' > '+LOCAL_PATH+'/hello'])
    
class ReadOp(dsl.ContainerOp):
    def __init__(self):
        super(ReadOp, self).__init__(
          name='read',
          image=BASE_IMAGE,
          command=['cat', LOCAL_PATH+'/hello'])

#### Define the pipeline
Pipeline function has to be decorated with the `@dsl.pipeline` decorator

In [6]:
@dsl.pipeline(
    name='pipeline volume mount',
    description='example to demonstrate volume mount')

def volume_mount(
        msg='Hello World!'
    ):
    write = WriteOp(msg).add_volume(k8s_client.V1Volume(name='workdir', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=PVC_NAME))).add_volume_mount(k8s_client.V1VolumeMount(mount_path=LOCAL_PATH, name='workdir'))
    read = ReadOp().after(write).add_volume(k8s_client.V1Volume(name='workdir', persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name=PVC_NAME))).add_volume_mount(k8s_client.V1VolumeMount(mount_path=LOCAL_PATH, name='workdir'))


#### Compile the pipeline

In [7]:
pipeline_func = volume_mount
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [8]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [9]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working